In [21]:
import os.path as osp

import pandas as pd

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

In [6]:
temp_dir = osp.join('.', 'temp')
data = pd.read_csv(osp.join(temp_dir, 'data.csv'), encoding='utf-8')
features = pd.read_csv(osp.join(temp_dir, 'features.csv'), encoding='utf-8')

In [9]:
kmean = KMeans(n_clusters=5)
clusters = kmean.fit_predict(features)

In [ ]:
test = pd.concat([data, pd.DataFrame(clusters, columns=['cluster'])], axis=1)
test.groupby(['topic', 'cluster']).count()

In [19]:
kmean = DBSCAN()
clusters = kmean.fit_predict(features)
pd.unique(clusters)

array([-1,  0,  2,  1,  3])

In [17]:
test = pd.concat([data, pd.DataFrame(clusters, columns=['cluster'])], axis=1)
test.groupby(['topic', 'cluster']).count()

In [22]:
kmean = AgglomerativeClustering(n_clusters=5)
clusters = kmean.fit_predict(features)
pd.unique(clusters)

array([1, 0, 4, 2, 3])

In [23]:
test = pd.concat([data, pd.DataFrame(clusters, columns=['cluster'])], axis=1)
test.groupby(['topic', 'cluster']).count()

article_index  link  article
topic     cluster                              
giao-duc  0                   52    52       52
          1                  372   372      372
          2                   11    11       11
          4                    6     6        6
khoa-hoc  0                  464   464      464
          1                   13    13       13
          2                   22    22       22
          3                    2     2        2
          4                    9     9        9
phap-luat 0                   95    95       95
          1                    3     3        3
          2                   13    13       13
          4                  284   284      284
suc-khoe  0                   50    50       50
          2                  493   493      493
          4                    4     4        4
the-thao  0                  138   138      138
          1                    1     1        1
          2                    6     6        6
          3                  391   391      391
          4                    3     3        3